# find and remove all duplicates
### We need to know if any company is a duplicate

In [2]:
packages = ['pymongo', 'gridfs', 'pandas']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
import pandas as pd
#endpoint = '10.0.10.5:27017' # use vpn
endpoint = '192.168.1.10:27017'
#endpoint = '192.168.229.55:27017'
database = 'supercias'
collection = 'companies'
# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
# get collection
collection = db[collection]
#find pdf file
companies_cursor = collection.find()

check if we have any duplicates left

In [3]:
pipeline = [
    {"$group": {
        "_id": "$ruc",  # Group by 'ruc' field
        "count": {"$sum": 1}  # Count occurrences of each 'rus' value
    }},
    {"$match": {
        "count": {"$gt": 1}  # Find only groups with more than 1 occurrence
    }}
]

# Execute the aggregation query
duplicates = list(collection.aggregate(pipeline))

# Output duplicate entries
if duplicates:
    print("Duplicate records found:")
    for doc in duplicates:
        print(f"ruc: {doc['_id']}, count: {doc['count']}")
else:
    print("No duplicate records found based on the 'ruc' field.")



No duplicate records found based on the 'ruc' field.


Merge all of the companies found, create a new object and delete all other ones.

In [25]:
# reset cursor
companies_cursor = collection.find()
# function that merges dictonaries
def merge_dicts(dicts):
    merged = {}
    for d in dicts:
        merged.update(d)
    return merged

# loop through all companies and get all unique rucs
companies_rucs = []
for comp in companies_cursor:
    if comp['ruc'] not in companies_rucs:
        companies_rucs.append(comp['ruc'])

# find all the companies duplicates which have the same ruc 
for ruc in companies_rucs:
    # query by ruc value
    query = collection.find({'ruc': ruc})
    # for every record in the query
    merged_record = {}
    for record in query: 
        # merge all the records
        merged_record = merge_dicts([record, merged_record])
        # delete the record from the collection
        collection.delete_one({'_id': record['_id']})
    # delete the _id from the merged record
    del merged_record['_id']
    # insert the merged record
    collection.insert_one(merged_record)

No Documentos online found
0990331014001
No Documentos online found
0991283765001
No Documentos online found
0190004937001
No Documentos online found
0190152588001
No Documentos online found
1791288246001
No Documentos online found
1791278410001
No Documentos online found
1791397398001
No Documentos online found
0990872511001
No Documentos online found
0992717955001
No Documentos online found
0990856125001
No Documentos online found
1790685411001
No Documentos online found
0992797827001
No Documentos online found
0791746051001
No Documentos online found
0992255412001
No Documentos online found
0490001026001
No Documentos online found
1791800486001
No Documentos online found
0790013360001
No Documentos online found
0190439569001
No Documentos online found
0992925825001
No Documentos online found
1792761557001
No Documentos online found
0990019606001
No Documentos online found
0992716525001
No Documentos online found
0993312029001
No Documentos online found
0992369922001
No Documentos on